Use the following Azure Databricks storage setup block only if you are using Azure Databricks. You can refer to the instructions here to get started:
https://docs.microsoft.com/en-us/azure/databricks/data/data-sources/azure/adls-gen2/azure-datalake-gen2-sp-access

If you are using Synapse Spark and if your data is residing on the storage attached to the Synapse Spark workspace, you can skip the below storage setup section.

In [ ]:
%scala

val storageAccountName = "<INSERT STORAGE ACCOUNT>"
val fileSystemName = "<INSERT CONTAINER NAME>"

val commonPath = "abfss://" + fileSystemName  + "@" + storageAccountName + ".dfs.core.windows.net"

# AAD Application Details
val appID = "<INSERT APP ID>"
val secret = "<INSERT SECRET>"
val tenantID = "<INSERT TENANT ID>"

spark.conf.set("fs.azure.account.auth.type." + storageAccountName + ".dfs.core.windows.net", "OAuth")
spark.conf.set("fs.azure.account.oauth.provider.type." + storageAccountName + ".dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set("fs.azure.account.oauth2.client.id." + storageAccountName + ".dfs.core.windows.net", "" + appID + "")
spark.conf.set("fs.azure.account.oauth2.client.secret." + storageAccountName + ".dfs.core.windows.net", "" + secret + "")
spark.conf.set("fs.azure.account.oauth2.client.endpoint." + storageAccountName + ".dfs.core.windows.net", "https://login.microsoftonline.com/" + tenantID + "/oauth2/token")
spark.conf.set("fs.azure.createRemoteFileSystemDuringInitialization", "true")
dbutils.fs.ls("abfss://" + fileSystemName  + "@" + storageAccountName + ".dfs.core.windows.net/")
spark.conf.set("fs.azure.createRemoteFileSystemDuringInitialization", "false")

In [ ]:
%scala

// Let us generate some parquet data first
import org.apache.spark.sql.{DataFrame, Row, SaveMode}
import org.apache.spark.sql.types.{StringType, IntegerType, StructField, StructType}

val tripsParquetPath = commonPath + "/parquet/trips"
val driverParquetPath = commonPath + "/parquet/driver"

// Generate sample trips data
val tripSchema = new StructType().add("tripId", StringType).add("driverId", StringType).add("customerId", StringType).add("cabId", StringType).add("tripDate", StringType).add("startLocation", StringType).add("endLocation", StringType)

val tripData = Seq(
  Row("100", "200", "300", "400", "20220101", "New York", "New Jersey"),
  Row("101", "201", "301", "401", "20220102", "Tempe", "Phoenix"),
  Row("102", "202", "302", "402", "20220103", "San Jose", "San Franciso"),
  Row("103", "203", "303", "403", "20220102", "New York", "Boston"),
  Row("104", "204", "304", "404", "20220103", "New York", "Washington"),
  Row("105", "205", "305", "405", "20220201", "Miami", "Fort Lauderdale"),
  Row("106", "206", "306", "406", "20220202", "Seattle", "Redmond"),
  Row("107", "207", "307", "407", "20220203", "Los Angeles", "San Diego"),
  Row("108", "208", "308", "408", "20220301", "Phoenix", "Las Vegas"),
  Row("109", "209", "309", "409", "20220302", "Washington", "Baltimore"),
  Row("110", "210", "310", "410", "20220303", "Dallas", "Austin"),
  Row("111", "211", "311", "411", "20220303", "New York", "New Jersey"),
  Row("112", "212", "312", "412", "20220304", "New York", "Boston"),
  Row("113", "212", "312", "412", "20220401", "San Jose", "San Ramon"),
  Row("114", "212", "312", "412", "20220404", "San Jose", "Oakland"),
  Row("115", "212", "312", "412", "20220404", "Tempe", "Scottsdale"),
  Row("116", "212", "312", "412", "20220405", "Washington", "Atlanta"),
  Row("117", "212", "312", "412", "20220405", "Seattle", "Portland"),
  Row("118", "212", "312", "412", "20220405", "Miami", "Tampa")
)

// Write Trips to Parquet
val tripWriteDF = spark.createDataFrame(spark.sparkContext.parallelize(tripData),tripSchema)
tripWriteDF.write.mode("overwrite").parquet(tripsParquetPath)

val driverSchema = new StructType().add("driverId", StringType).add("name", StringType).add("license",StringType).add("gender",StringType).add("salary",IntegerType)

val driverData = Seq(
  Row("200", "Alice", "A224455", "Female", 3000),
  Row("202", "Bryan","B992244","Male",4000),
  Row("204", "Catherine","C887733","Female",4000),
  Row("208", "Daryl","D229988","Male",3000),
  Row("212", "Jenny","J663300","Female", 5000)
)
// Write Driver to Parquet
val driverWriteDF = spark.createDataFrame(spark.sparkContext.parallelize(driverData),driverSchema)
driverWriteDF.write.mode("overwrite").parquet(driverParquetPath)

In [ ]:
%python
# Let us read the data from Parquet files and view them as a Dataframe using Python and SQL now

storageAccountName = "<INSERT STORAGE ACCOUNT>"
fileSystemName = "<INSERT CONTAINER NAME>"
commonPath = "abfss://" + fileSystemName  + "@" + storageAccountName + ".dfs.core.windows.net"

df = spark.read.load( commonPath + '/parquet/trips/*.parquet',  format='parquet')
df.printSchema()

spark.sql("DROP DATABASE TripsDatabase")

spark.sql("CREATE DATABASE IF NOT EXISTS TripsDatabase")
df.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable("TripsTable")

sqldf = spark.sql("SELECT * FROM TripsTable") 
display(sqldf)



In [ ]:
df = spark.read.load( commonPath + '/parquet/trips/*.parquet',  format='parquet')
spark.sql("CREATE DATABASE IF NOT EXISTS TripsDatabase")
df.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable("TripsTable")
sqldf = spark.sql("""
   SELECT COUNT(*) AS Trips, 
   startLocation AS Location 
   FROM TripsTable 
   GROUP BY startLocation """) 
display(sqldf)